In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium

In [3]:
CLIENT_ID = 'UC5JPATLD4ZDRIFLB31QMJHIM3CVY1XM2J3FALJQ0J15BO2U' # your Foursquare ID
CLIENT_SECRET = 'U5YHMLDXRKITTZ3C15CBUFPQ20IJ05H54N2Z4MSV1WPZA5WB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UC5JPATLD4ZDRIFLB31QMJHIM3CVY1XM2J3FALJQ0J15BO2U
CLIENT_SECRET:U5YHMLDXRKITTZ3C15CBUFPQ20IJ05H54N2Z4MSV1WPZA5WB


In [4]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "sea",
    "near": "jeddah",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [5]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [6]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Jeddah', 'Jeddah')

In [7]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 21.549953769425866, 'lng': 39.20764788955189},
  'sw': {'lat': 21.532652188041883, 'lng': 39.18885686240229}},
 46)

In [8]:
d["geocode"]

{'what': '',
 'where': 'jeddah',
 'center': {'lat': 21.54238, 'lng': 39.19797},
 'displayString': 'Jeddah, Saudi Arabia',
 'cc': 'SA',
 'geometry': {'bounds': {'ne': {'lat': 21.817041, 'lng': 39.303589},
   'sw': {'lat': 21.314672, 'lng': 39.047565}}},
 'slug': 'jeddah',
 'longId': '72057594038033279'}

In [9]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [10]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [11]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 46


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4f367de1e4b08f00986ceee5',
  'name': 'العماري للأقمشة',
  'location': {'address': 'East comm. Ctr. 1 gate 7',
   'crossStreet': 'Makarona St',
   'lat': 21.542388997135777,
   'lng': 39.197059467440354,
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.542388997135777,
     'lng': 39.197059467440354}],
   'postalCode': '21382',
   'cc': 'SA',
   'city': 'جدة',
   'state': 'منطقة مكة',
   'country': 'المملكة العربية السعودية',
   'formattedAddress': ['East comm. Ctr. 1 gate 7 (Makarona St)',
    'جدة 21382',
    'المملكة العربية السعودية']},
  'categories': [{'id': '4bf58dd8d48988d108951735',
    'name': "Women's Store",
    'pluralName': "Women's Stores",
    'shortName': "Women's Store",
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_women_',
     'suffix': '.png'},
    'primary': True}],

In [12]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '505f1bd2e4b07e3a197acd9f',
  'name': 'Montana Sweets (حلواني مونتانا)',
  'location': {'lat': 21.542706002985202,
   'lng': 39.19941143029486,
   'labeledLatLngs': [{'label': 'display',
     'lat': 21.542706002985202,
     'lng': 39.19941143029486}],
   'cc': 'SA',
   'country': 'المملكة العربية السعودية',
   'formattedAddress': ['المملكة العربية السعودية']},
  'categories': [{'id': '4bf58dd8d48988d1bc941735',
    'name': 'Cupcake Shop',
    'pluralName': 'Cupcake Shops',
    'shortName': 'Cupcakes',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cupcakes_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-505f1bd2e4b07e3a197acd9f-1'}

In [13]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i theaters" % len(df))
df.head()

{'address': 'East comm. Ctr. 1 gate 7', 'crossStreet': 'Makarona St', 'lat': 21.542388997135777, 'lng': 39.197059467440354, 'labeledLatLngs': [{'label': 'display', 'lat': 21.542388997135777, 'lng': 39.197059467440354}], 'postalCode': '21382', 'cc': 'SA', 'city': 'جدة', 'state': 'منطقة مكة', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['East comm. Ctr. 1 gate 7 (Makarona St)', 'جدة 21382', 'المملكة العربية السعودية']}
{'lat': 21.542706002985202, 'lng': 39.19941143029486, 'labeledLatLngs': [{'label': 'display', 'lat': 21.542706002985202, 'lng': 39.19941143029486}], 'cc': 'SA', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['المملكة العربية السعودية']}
{'lat': 21.544912301906813, 'lng': 39.194185210027236, 'labeledLatLngs': [{'label': 'display', 'lat': 21.544912301906813, 'lng': 39.194185210027236}], 'cc': 'SA', 'city': 'جدة', 'state': 'منطقة مكة', 'country': 'المملكة العربية السعودية', 'formattedAddress': ['جدة', 'المملكة العربية السعودية']}
{'address': 'شارع 

,uid,name,shortname,address,postalcode,lat,lng
0,4f367de1e4b08f00986ceee5,العماري للأقمشة,Women's Store,,21382,21.542389,39.197059
1,4dad957a6e81d745d7446f6b,Mobily Office (شارع غرناطة),Office,,11423,21.545246,39.195517
2,5bc31c5d9cadd9002c93b36a,Fitness Time Ladies,Gym / Fitness,,23335,21.533439,39.196927
3,5e2bfa867ba3780008492ff4,Dunkin Donuts,Donuts,,23333,21.546928,39.189711
4,5d1b568db55cb800239e28c5,Starbucks,Coffee Shop,,23335,21.533474,39.196891


In [14]:
Jeddah_center = d["geocode"]["center"]
Jeddah_center

{'lat': 21.54238, 'lng': 39.19797}

In [15]:
df.head(20)

,uid,name,shortname,address,postalcode,lat,lng
0,4f367de1e4b08f00986ceee5,العماري للأقمشة,Women's Store,,21382,21.542389,39.197059
1,4dad957a6e81d745d7446f6b,Mobily Office (شارع غرناطة),Office,,11423,21.545246,39.195517
2,5bc31c5d9cadd9002c93b36a,Fitness Time Ladies,Gym / Fitness,,23335,21.533439,39.196927
3,5e2bfa867ba3780008492ff4,Dunkin Donuts,Donuts,,23333,21.546928,39.189711
4,5d1b568db55cb800239e28c5,Starbucks,Coffee Shop,,23335,21.533474,39.196891
5,5d94c741f3b98d0009009205,Dunkin Dounat,Donuts,,23333,21.546753,39.189772
6,58126ff138fa9eb4bd5f6688,Abu Zaid Restaurant (مطاعم أبو زيد),Diner,,23335,21.534021,39.201100
7,4f6f7d22e4b0e3b3bed8b363,Ashwaq Resturant (مطعم أشواق),Middle Eastern,,23337,21.541838,39.190843
8,4d6fb08e56dea0936c66c38c,Al Sharq Shopping Center (مجمع الشرق التجاري),Women's Store,,21332,21.543123,39.197062
9,4f0c3e8ce4b0261c93fb881d,مطابخ ومطاعم السرور,Middle Eastern,,321456,21.549167,39.194904


In [16]:
from folium import plugins


map_jeddah = folium.Map(location=[21.54238, 39.19797], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_jeddah)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_jeddah.add_child(plugins.HeatMap(hm_data))

map_jeddah

In [17]:
lat = 21.54238
lng = 39.19797
map_jeddah = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_jeddah)
map_jeddah